# BATCH GRADIENT DESCENT IMPLEMENTATION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## LIBRARY IMPORT

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras

## DATA IMPORT

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/03_GRADIENT_DESCENT_NN/05_insurance_data.csv')

In [4]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
df.shape

(28, 3)

## TRAIN TEST SPLIT

In [6]:
x_train,x_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size = 0.2, random_state = 99)

In [7]:
len(x_train)

22

## SCALING THE AGE

In [8]:
x_train_scale = x_train.copy()
x_train_scale['age'] = x_train_scale['age']/100

x_test_scale = x_test.copy()
x_test_scale['age'] = x_test_scale['age']/100

In [9]:
x_train_scale

,age,affordibility
10,0.18,1
15,0.55,1
2,0.47,1
7,0.60,0
20,0.21,1
14,0.49,1
6,0.55,0
11,0.28,1
13,0.29,0
0,0.22,1


## BUILD SIMPLE NN

In [10]:
# model = keras.Sequential([
#     keras.layers.Dense(1,input_shape = (2,),activation = 'sigmoid',
#               kernel_initializer = 'ones',
#               bias_initializer = 'zeros')
# ])
# model.compile(
#     optimizer = 'adam',
#     loss = 'binary_crossentropy',
#     metrics = ['accuracy']
# )
# model.fit(x_train_scale,y_train,epochs = 5000)

In [11]:
#model.evaluate(x_test_scale,y_test)

In [16]:
#model.predict(x_test_scale)

In [20]:
# coef,intercept  = model.get_weights()
# coef,intercept

In [13]:
x_test_scale

,age,affordibility
17,0.58,1
22,0.40,1
16,0.25,0
12,0.27,0
19,0.18,1
23,0.45,1


# RAW NN FROM SCRATCH

In [21]:
# coef,intercept  = model.get_weights() from the above NN model
coef,intercept = np.array([[4.9306455],
        [1.6307154]]),np.array([-3.2907383])

In [19]:
def sigmoid(x):
  import math
  return 1/ (1+ math.exp(-x))

In [22]:
sigmoid(18)

0.9999999847700205

In [23]:
def prediction_function(age,affordibility):
  # lets make the formula of weighted sum
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)

In [24]:
prediction_function(0.58,1) #from x_test

/tmp/ipython-input-19-786199062.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/ (1+ math.exp(-x))


0.7684805720015184

In [25]:
prediction_function(0.18,1)

/tmp/ipython-input-19-786199062.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/ (1+ math.exp(-x))


0.31593710417504595

## GRADIENT DESCENT FROM SCRATCH

In [26]:
def log_loss(y_true,y_pred):
  epsilon = 1e-15
  y_pred_new = [max(i,epsilon) for i in y_pred]
  y_pred_new = [min(i,1-epsilon) for i in y_pred_new]
  y_pred_new = np.array(y_pred_new)
  return -np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))

In [27]:
def sigmoid_numpy(x):
  return 1/ (1+ np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [28]:
def gradient_descent(age,affordibility,y_true,epochs,loss_threshold):
  w1 = w2 = 1
  bias = 0
  rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = w1*age + w2*affordibility +bias
    y_pred = sigmoid_numpy(weighted_sum)

    loss = log_loss(y_true,y_pred)

    w1d = (1/n)* np.dot(np.transpose(age),(y_pred-y_true))
    w2d = (1/n)* np.dot(np.transpose(affordibility),(y_pred-y_true))

    bias_d = np.mean(y_pred - y_true)

    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias - rate * bias_d

    print(f'Epochs : {i}, w1 : {w1},w2 : {w2},Bias : {bias},loss : {loss}')

    if loss <= loss_threshold:
      break
  return w1,w2,bias

# 0.3853 is the loss from my tensorflow model

TENSORFLOW WEIGHTS AND BIAS:  
W1 = 4.9306455  
W2 = 1.6307154  
BIAS = -3.2907883  

RAW MODEL OF GRADIENT DESCENT:  
W1 = 4.832322774622927  
W2 = 1.8140488973450755   
BIAS = -3.442403858447309  


In [40]:
class myNN():
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0
  def fit(self,x,y,epochs,loss_threshold):
    self.w1,self.w2,self.bias = self.gradient_descent(x['age'],x.affordibility,y,epochs,loss_threshold)
  def predict(self,x_test):
    weighted_sum = self.w1 * x_test['age']+ self.w2 * x_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)
  def gradient_descent(self,age,affordibility,y_true,epochs,loss_threshold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
      weighted_sum = w1*age + w2*affordibility +bias
      y_pred = sigmoid_numpy(weighted_sum)

      loss = log_loss(y_true,y_pred)

      w1d = (1/n)* np.dot(np.transpose(age),(y_pred-y_true))
      w2d = (1/n)* np.dot(np.transpose(affordibility),(y_pred-y_true))

      bias_d = np.mean(y_pred - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * bias_d

      if i%50 == 0:
        print(f'Epochs : {i}, w1 : {w1},w2 : {w2},Bias : {bias},loss : {loss}')

      if loss <= loss_threshold:
         print(f'Epochs : {i}, w1 : {w1},w2 : {w2},Bias : {bias},loss : {loss}')
         break
    return w1,w2,bias

In [41]:
custom_model = myNN()
custom_model.fit(x_train_scale,y_train,epochs = 500, loss_threshold = 0.3853)

Epochs : 0, w1 : 0.9679458843047224,w2 : 0.9300842621913651,Bias : -0.1415772425202991,loss : 0.7530551024331085
Epochs : 50, w1 : 1.6307083585422055,w2 : 1.1723437052165167,Bias : -1.556943856419746,loss : 0.527389867833077
Epochs : 100, w1 : 2.496545984604332,w2 : 1.470453427980084,Bias : -2.1669567817015767,loss : 0.47844412113289897
Epochs : 150, w1 : 3.309749095082006,w2 : 1.6306199145264593,Bias : -2.6439277819097753,loss : 0.4416679244944602
Epochs : 200, w1 : 4.061608469750626,w2 : 1.7316422752849194,Bias : -3.046622535531895,loss : 0.41204421234662225
Epochs : 250, w1 : 4.7531811883069155,w2 : 1.806108673475393,Bias : -3.4021650501537617,loss : 0.38755102728951024
Epochs : 256, w1 : 4.832322774622927,w2 : 1.8140488973450755,Bias : -3.442403858447309,loss : 0.384895539677972


In [42]:
custom_model.predict(x_test_scale)

,0
17,0.763938
22,0.575560
16,0.096710
12,0.105488
19,0.318964
23,0.633251
